### Вам нужно предсказать цену товара по другим характеристикам.

Метрика качества RMSLE.

can_buy - возможно ли купить этот товар
can_promote - возможность продвигать товар,
category - категория товара,
contacts_visible - видны ли контакты,
date_created - таймстемп создания товара,
delivery_available - возможность доставки,
description - описание,
fields - свойства товара,
id - идентификатор продукта,
images - id картинок,
location - местрорасположение,
mortgage_available - возможность ипотеки,
name - навзаник,
payment_available - возможность онлайн оплаты,
price - цена (в копейках),
subcategory - подкатегория,
subway - метро,

Цена в КОПЕЙКАХ Но предсказания нужны в РУБЛЯХ, потому - сразу поделите в train прайс на 100

Файлы
train_4_col_2.csv.zip - кусок TRAIN, который содержит id, category, subcategory (строки все)
test_3_col_2.csv.zip - кусок TEST, который содержит id, category, subcategory (строки все)

train_4_col_3.csv.zip - кусок TRAIN, который содержит id, fields (строки все)
test_3_col_3.csv.zip - кусок TEST, который содержит id, fields (строки все)

train_sample.pckl.zip - кусок TRAIN, который содержит все поля, но только первые 100000 строк

train_4_col.csv.zip - кусок TRAIN, который содержит id, name, description, price (строки все)
test_3_col.csv.zip - кусок TEST, который содержит id, name, description (строки все)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

from bs4 import BeautifulSoup 
import pymorphy2, re, os, time
import pymystem3
from datetime import datetime
import re
import ast

from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## Feature Extraction

In [5]:
%%time
df_train_1 = pd.read_csv('train_4_col.csv.zip', compression='zip', delimiter='\t', encoding='utf8', index_col=0)
df_test_1 = pd.read_csv('test_3_col.csv.zip', compression='zip', delimiter='\t', encoding='utf8', index_col=0)

df_train_2 = pd.read_csv('train_4_col_2.csv.zip', compression='zip', delimiter='\t', encoding='utf8', index_col=0)
df_train_2 = df_train_2.drop('id', axis=1)
df_test_2 = pd.read_csv('test_3_col_2.csv.zip', compression='zip', delimiter='\t', encoding='utf8', index_col=0)
df_test_2 = df_test_2.drop('id', axis=1)

df_train_3 = pd.read_csv('train_4_col_3.csv.zip', compression='zip', delimiter='\t', encoding='utf8', index_col=0)
df_train_3 = df_train_3.drop('id', axis=1)
df_test_3 = pd.read_csv('test_3_col_3.csv.zip', compression='zip', delimiter='\t', encoding='utf8', index_col=0)
df_test_3 = df_test_3.drop('id', axis=1)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 49.5 s, sys: 4.62 s, total: 54.2 s
Wall time: 54.7 s


In [6]:
sample = pd.read_csv('submit_Sample.csv', delimiter='\t', encoding='utf8', index_col=0)
sample.head()

""
"id,price"
"285ea2e9935ccdeb8378c6a5,0"
"adfb73820bbb831257df6e95,0"
"783025601c36202f633fc6a5,0"
"2f0cd2d2e15dc90afd847f95,0"
"5c23a37902855a20172845a5,0"


In [21]:
df_test_1.head()

,id,name,description
4,285ea2e9935ccdeb8378c6a5,Стремянка,"Стремянка трехсекционная,3×4"
9,adfb73820bbb831257df6e95,Плита электрическая,Плита ЭВИ 5120. Работают 3 конфорки и духовка....
15,783025601c36202f633fc6a5,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,Все лоты привезены с площадок и аукционов Япон...
19,2f0cd2d2e15dc90afd847f95,Batman Archam Knight,NaN
20,5c23a37902855a20172845a5,Ледобур для зимней рыбалки,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...


In [7]:
df_train_2.head()

,category,subcategory
0,9,914
1,22,2202
2,22,2204
3,22,2209
5,22,2204


In [8]:
df_train_3.head()

,fields
0,"[{'field': {'name': 'Женский гардероб', 'id': ..."
1,"[{'field': {'name': 'Детский гардероб', 'id': ..."
2,"[{'field': {'name': 'Детский гардероб', 'id': ..."
3,"[{'field': {'name': 'Детский гардероб', 'id': ..."
5,"[{'field': {'name': 'Детский гардероб', 'id': ..."


In [15]:
df_train = pd.concat([df_train_1, df_train_2, df_train_3], axis=1)
df_train['id_'] = df_train.index
df_test = pd.concat([df_test_1, df_test_2, df_test_3], axis=1)
df_test['id_'] = df_test.index

print(df_train.shape)
print(df_test.shape)

df_train.info()

(1748890, 8)
(749525, 7)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1748890 entries, 0 to 99930
Data columns (total 8 columns):
id             object
name           object
description    object
price          int64
category       int64
subcategory    int64
fields         object
id_            int64
dtypes: int64(4), object(4)
memory usage: 120.1+ MB


In [16]:
df_train.head()

,id,name,description,price,category,subcategory,fields,id_
0,3edeb34cf93f490ff760af85,Сумка DG,NaN,199900,9,914,"[{'field': {'name': 'Женский гардероб', 'id': ...",0
1,c98febd50dad3cc0ffc86085,Комплект,8-12 лет,35000,22,2202,"[{'field': {'name': 'Детский гардероб', 'id': ...",1
2,ade01e13912a46a99134cc75,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",30000,22,2204,"[{'field': {'name': 'Детский гардероб', 'id': ...",2
3,ab3e6941c11304c1519aef75,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",80000,22,2209,"[{'field': {'name': 'Детский гардероб', 'id': ...",3
5,252452a91c944a22c276d995,Жилет теплый,NaN,50000,22,2204,"[{'field': {'name': 'Детский гардероб', 'id': ...",5


In [17]:
df_test.head()

,id,name,description,category,subcategory,fields,id_
4,285ea2e9935ccdeb8378c6a5,Стремянка,"Стремянка трехсекционная,3×4",6,603,"[{'field': {'name': 'Ремонт и строительство', ...",4
9,adfb73820bbb831257df6e95,Плита электрическая,Плита ЭВИ 5120. Работают 3 конфорки и духовка....,2,203,"[{'field': {'name': 'Бытовая техника', 'id': 2...",9
15,783025601c36202f633fc6a5,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,Все лоты привезены с площадок и аукционов Япон...,1,116,"[{'field': {'name': 'Транспорт и автотовары', ...",15
19,2f0cd2d2e15dc90afd847f95,Batman Archam Knight,NaN,10,1009,"[{'field': {'name': 'Хобби и развлечения', 'id...",19
20,5c23a37902855a20172845a5,Ледобур для зимней рыбалки,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...,11,1104,"[{'field': {'name': 'Спорт и отдых', 'id': 11,...",20


In [2]:
def parseFields(fields):
    res_list = list()
    for node in ast.literal_eval(fields):
        if type(node) == dict:
            field = node['field']
        else:
            field = node[0]['field']
        ftype = field['type'] 
        if ftype == 'category' or ftype == 'subcategory':
            res_list.append(field['name'])     
    return ' '.join(res_list) 

def parseAddFields(fields):
    res_list = list()
    for node in ast.literal_eval(fields):
        if type(node) == dict:
            field = node['field']
        else:
            field = node[0]['field']
        ftype = field['type'] 
        if ftype == 'additional_field':
            if len(node['values']) > 0:
                res_list.append(field['name'] + ' ' + node['values'][0]['selected_value']['value'])
            else:
                res_list.append(field['name'])      
    return ' '.join(res_list) 

In [6]:
def cleanText(text):
    res = re.sub('[()/|,.:;–!?]',' ', str(text))
    #res = re.sub('[^а-яёА-ЯЁa-zA-Z ]', ' ', res)
    res = ' '.join(word for word in res.split() if len(word)>3)
    res = re.sub(r'\s+', ' ', res)
    return res.lower()

cachedDict = dict()
morph = pymorphy2.MorphAnalyzer()
def parseMorph(text, cached=True):
    resultList = []
    wordList = re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', str(text))
    for word in wordList:

        if cached & (word in cachedDict):
            resultList.append(cachedDict[word])
        else:
            pv = morph.parse(word)
            for p in pv:
                if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    resultList.append(p.normal_form)
                    if cached:
                        cachedDict[word] = p.normal_form
                    break
    return resultList

mystem=pymystem3.Mystem()
def parseMystem(text):
    resultList = []
    trashList = ['', '\n', '\t']
    wordsList = mystem.lemmatize(text)
    for word in wordsList:
        if word.strip() not in trashList:
            resultList.append(word)
    return resultList

def preprocData(df_input, file_suffix = 'data'):
    #df_output = df_input[:100].copy()
    df_output = df_input.copy()
        
    print('Parse fields...')
    start_time = time.time()
    df_output['fields_categs'] = df_output['fields'].map(parseFields)
    df_output['fields_adds'] = df_output['fields'].map(parseAddFields)
    #df_output['fields_str'] = df_output['fields'].map(lambda x: ' '.join(x))
    df_output = df_output.drop(['fields'], axis=1)
    print('Wait time {:.2f} s'.format((time.time()-start_time))) 
    
    print('Чистка текста...')
    start_time = time.time()
    df_output['name'] = df_output['name'].fillna('')
    df_output['description'] = df_output['description'].fillna('')
    df_output['text'] = df_output['name'] + ' ' + df_output['description']
    df_output['text'] = df_output['text'].map(cleanText)
    print('Wait time {:.2f} s'.format((time.time()-start_time)))
    
    print('Морфоанализ pymorphy2...')
    start_time = time.time()    
    df_output['text_morph'] = df_output['text'].map(parseMorph)
#    df_output['text_morph_str'] = df_output['text_morph'].map(lambda x: ' '.join(x))
#    df_output = df_output.drop(['fields'], axis=1)
    print('Wait time {:.2f} s'.format((time.time()-start_time)))
    
    if file_suffix == 'train':
        print('Перевожу копейки в рубли')
        start_time = time.time()
        df_output['price'] = df_output['price']*100
        print('Wait time {:.2f} s'.format((time.time()-start_time)))
    
    #print('Удаление ненужных полей...')
    #dropList = ['category', 'subcategory', 'date_created', 'description', 'fields', 'images', 'location', 'name', 'subway', 'fields_str']
    #df_output = df_output.drop(dropList, axis=1)
    #print('Wait time {:.2f} s'.format((time.time()-start_time)))
       
    #df_output = df_output.drop(['text_list'], axis=1)
    
    print('Сохраняю результат вычислений...')
    start_time = time.time()
    fileName = file_suffix + '_preproc_' + datetime.now().strftime('%d%m%y_%H%M') + '.csv'
    df_output.to_csv(fileName, sep=',', encoding='utf-8', index=False)
    print('Wait time {:.2f} s'.format((time.time()-start_time)))
    print('Создан файл', fileName)

    #print('Морфоанализ pymystem3...')
    #start_time = time.time()
    #df_output['text_mystem'] = df_output['text'].map(parseMystem)
    #df_output['text_mystem_str'] = df_output['text_mystem'].map(lambda x: ' '.join(x))
    #print('Wait time {:.2f} s'.format((time.time()-start_time)))
    
    print(df_output.shape)
    print('Done\n')

    return df_output

In [9]:
%%time
# Обработка данных с сохранением результата в файл
#df_train_preproc= preprocData(df_train, 'train')
#df_test_preproc= preprocData(df_test, 'test')

df_train_preproc= preprocData(df_train_preproc, 'train')
df_test_preproc= preprocData(df_test_preproc, 'test')

os.system('say "Обработка данных завершена!"')

Морфоанализ pymorphy2...
Wait time 653.33 s
Перевожу копейки в рубли
Wait time 0.67 s
(1748890, 11)
Done

Морфоанализ pymorphy2...
Wait time 369.85 s
(749525, 10)
Done

CPU times: user 14min 36s, sys: 37.3 s, total: 15min 14s
Wall time: 17min 25s


In [3]:
%%time
# Востанавление тренировочных данных из файла
df_train_preproc = pd.read_csv('train_preproc_091118_1218.csv')
print(df_train_preproc.shape)

# Востанавление тестовых данных из файла
df_test_preproc = pd.read_csv('test_preproc_091118_1229.csv')
print(df_test_preproc.shape)

os.system('say "Загрузка данных завершена!"')

(1748890, 11)
(749525, 10)
CPU times: user 27.7 s, sys: 5.93 s, total: 33.6 s
Wall time: 37.9 s


In [7]:
df_train_preproc

,id,name,description,price,category,subcategory,id_,fields_categs,fields_adds,text,text_morph
0,3edeb34cf93f490ff760af85,Сумка DG,NaN,1999.0,9,914,0,Женский гардероб Аксессуары,Тип Сумки Цвет Черный,сумка,['сумка']
1,c98febd50dad3cc0ffc86085,Комплект,8-12 лет,350.0,22,2202,1,Детский гардероб Аксессуары,Тип Пол Цвет,комплект 8-12,['комплект']
2,ade01e13912a46a99134cc75,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",300.0,22,2204,2,Детский гардероб Верхняя одежда,Тип Пол Размер (рост) Цвет Сезон,пальтишко демисезонное девочку состояние хорошее,"['пальтишко', 'демисезонный', 'девочка', 'сост..."
3,ab3e6941c11304c1519aef75,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",800.0,22,2209,3,Детский гардероб Обувь,Тип Размер Пол Цвет Сезон,attipas размер-135mm euro-22 прочная мягкая ск...,"['размер', 'прочный', 'мягкий', 'скользящий', ..."
4,252452a91c944a22c276d995,Жилет теплый,NaN,500.0,22,2204,5,Детский гардероб Верхняя одежда,Тип Пол Размер (рост) Цвет Сезон,жилет теплый,"['жилет', 'тёплый']"
5,364f7659a4e9ba6c53e35495,"Босоножки, сабо","Новые, текстильные босоножки, 37 размер в разм...",300.0,9,902,6,Женский гардероб Обувь,Тип Размер Цвет Сезон,босоножки сабо новые текстильные босоножки раз...,"['босоножка', 'сабо', 'новое', 'текстильный', ..."
6,c3d87065e4dbc080f400bb85,Жесткий диск для ноутбука Seagate 320gb,Seagate Модель St320lt020 HDD для ноутбуков ...,1500.0,15,1508,7,Компьютерная техника Комплектующие и запчасти,Тип Устройство,жесткий диск ноутбука seagate 320gb seagate мо...,"['жёсткий', 'диск', 'ноутбук', 'модель', 'ноут..."
7,22e6a518d53f490f6ce7fd85,Ковер 324 на 240,"Состояние отличное,без дефектов",2000.0,4,403,8,Для дома и дачи Текстиль и ковры,Тип Ковры,ковер состояние отличное дефектов,"['ковёр', 'состояние', 'отличный', 'дефект']"
8,c6bd550b7f1304c1f7fda485,Кенгуру,"Продам кенгуру 350 , ванночка с горкой 250",350.0,3,301,10,Детские товары Другое,NaN,кенгуру продам кенгуру ванночка горкой,"['кенгуру', 'продать', 'кенгуру', 'ванночка', ..."
9,117b885a3cf9554042ae2285,Машинка автомат,NaN,3500.0,2,208,11,Бытовая техника Стиральные машины,Тип Марка,машинка автомат,"['машинка', 'автомат']"


In [8]:
df_test_preproc

,id,name,description,category,subcategory,id_,fields_categs,fields_adds,text,text_morph
0,285ea2e9935ccdeb8378c6a5,Стремянка,"Стремянка трехсекционная,3×4",6,603,4,Ремонт и строительство Ручные инструменты,Тип,стремянка стремянка трехсекционная,"['стремянка', 'стремянка', 'трехсекционный']"
1,adfb73820bbb831257df6e95,Плита электрическая,Плита ЭВИ 5120. Работают 3 конфорки и духовка....,2,203,9,Бытовая техника Плиты и духовые шкафы,Тип,плита электрическая плита 5120 работают конфор...,"['плита', 'электрический', 'плита', 'работать'..."
2,783025601c36202f633fc6a5,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,Все лоты привезены с площадок и аукционов Япон...,1,116,15,Транспорт и автотовары Диски,"Тип Литые Ширина обода, дюймов 7 Диаметр, дюйм...",диски rays mazdaspeed touring 5х114 лоты приве...,"['диск', 'лот', 'площадка', 'аукцион', 'япония..."
3,2f0cd2d2e15dc90afd847f95,Batman Archam Knight,NaN,10,1009,19,Хобби и развлечения Игры для приставок и ПК,Платформа PlayStation 4 Жанр Драки,batman archam knight,[]
4,5c23a37902855a20172845a5,Ледобур для зимней рыбалки,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...,11,1104,20,Спорт и отдых Охота и рыбалка,Тип Ледобуры,ледобур зимней рыбалки отдаю ледобур времён ди...,"['ледобур', 'зимний', 'рыбалка', 'отдавать', '..."
5,7b5ee55fc6ab657218c315a5,Sony плэйстейшен 2,Сонька в идеальном состоянии. 9 Дисков: CALL o...,10,1002,21,Хобби и развлечения Игровые приставки,Тип Приставки Платформа PlayStation 2,sony плэйстейшен сонька идеальном состоянии ди...,"['плэйстейшня', 'сонька', 'идеальный', 'состоя..."
6,7f2526891b18fff0b613e2a5,iPhone 4s 16Gb,"На запчасти или под восстановление, иногда син...",12,1202,24,Телефоны и планшеты Мобильные телефоны,Тип Марка Apple Операционная система iOS Диаго...,iphone 16gb запчасти восстановление иногда син...,"['запчасть', 'восстановление', 'синеть', 'проп..."
7,8a36445a546c1e26657922a5,Манеж,Компактный удобный складывается в виде трости ...,3,317,28,Детские товары Обустройство детской,Тип Манежи,манеж компактный удобный складывается виде тро...,"['манеж', 'компактный', 'удобный', 'складывать..."
8,8307b793f4855a2010378f95,Оригинальный костюм жакет и юбка Olar Collect....,Продам костюм фирмы OLAR Collection S-44 рос. ...,9,908,29,Женский гардероб Пиджаки и костюмы,Тип Костюмы с юбкой Размер 42-44 (S) Цвет Серый,оригинальный костюм жакет юбка olar collect пр...,"['оригинальный', 'костюм', 'жакет', 'юбка', 'п..."
9,b564c2b6763f490fe8fbb7a5,Теплые зимние сапоги 37 размер,Состояние очень хорошее.,9,902,31,Женский гардероб Обувь,Тип Сапоги Размер 37 Цвет Черный Сезон Зима,теплые зимние сапоги размер состояние очень хо...,"['тёплый', 'зимний', 'сапог', 'размер', 'состо..."


In [16]:
from collections import Counter

def printTop(data, top=100):
    i = 0
    for e in sorted(data.items(), key=lambda x:x[1], reverse=True):
        print(str(i)+':',e[0], e[1], end=' ')
        i += 1
        if i > top: break 

def countDict(articleList, articleList2):
    resDict = dict()
    for article in articleList:
        wordSet = set(article) # Избавляемся от дубликатов, неважно сколько раз слово встречалось в одной статье
        for word in wordSet:
            resDict[word] = resDict.get(word, 0) + 1
    for article in articleList2:
        wordSet = set(article) # Избавляемся от дубликатов, неважно сколько раз слово встречалось в одной статье
        for word in wordSet:
            resDict[word] = resDict.get(word, 0) + 1
    return resDict

In [17]:
#df_train_preproc['text_list'] = df_train_preproc['text'].map(lambda x: x.split())
#df_test_preproc['text_list'] = df_test_preproc['text'].map(lambda x: x.split())
text_cnt = countDict(df_train_preproc['text_morph'], df_test_preproc['text_morph'])
printTop(text_cnt)

0: состояние 698387 1: размер 410498 2: новый 343008 3: хороший 311052 4: отличный 288368 5: продать 287864 6: новое 227951 7: быть 221695 8: торг 194424 9: комплект 174985 10: продавать 159616 11: один 131499 12: цена 131237 13: детский 120734 14: работать 110094 15: идеальный 108805 16: телефон 108033 17: год 98333 18: отдать 98280 19: платье 95569 20: фото 93860 21: чехол 86257 22: цвета 84615 23: зимний 84250 24: подарок 84185 25: тёплый 81692 26: обмен 81149 27: коробка 81097 28: подойти 80555 29: рабочий 79010 30: девочка 75618 31: ребёнок 71020 32: профиль 66808 33: месяц 66136 34: покупать 64969 35: куртка 63752 36: весь 63526 37: удобный 63491 38: наличие 58051 39: мочь 57690 40: самовывоз 57256 41: кожа 56048 42: натуральный 55922 43: раз 54392 44: полный 54295 45: длина 54091 46: рост 53489 47: пользоваться 52553 48: сумка 51814 49: пара 51494 50: мальчик 50655 51: продаваться 50541 52: диск 50287 53: красивый 50060 54: высота 49549 55: сантиметр 49150 56: женский 48399 57: 

In [18]:
def getStopWords(wordsList, limit=0):
    result = list()
    i = 0
    for e in sorted(wordsList.items(), key=lambda x:x[1], reverse=True):
        if i >= limit: break
        result.append(str(e[0]).lower())
        i += 1
    return result
        
stop_words = getStopWords(text_cnt, 16)

print(stop_words)

['состояние', 'размер', 'новый', 'хороший', 'отличный', 'продать', 'новое', 'быть', 'торг', 'комплект', 'продавать', 'один', 'цена', 'детский', 'работать', 'идеальный']


### Добавляем города

In [ ]:
df_loc_train = pd.read_pickle('train_hack.pckl')
df_loc_test = pd.read_pickle('test_hack.pckl')

In [ ]:
df_locations =  pd.concat([
    df_loc_train.DataFrame(df_loc_train[['location', 'id']]), 
    df_loc_test.DataFrame(df_loc_test[['location', 'id']])
])

In [ ]:
import reverse_geocoder as rg

def reverseGeocoder(df_input):
    
    df_output = df_input.copy()
    
    df_output['location'] = df_output['location'].map(lambda x: ast.literal_eval(str(x)))
    df_output['coordinates'] = df_output['location'].map(lambda x: (x['latitude'], x['longitude']))
    df_output['result'] = rg.search(df_output['coordinates'].tolist())
    df_output['city'] = df_output['result'].map(lambda x: ast.literal_eval(str(x['admin1']))) 
    
    return df_output

In [ ]:
df_locations_preproc = reverseGeocoder(df_locations)

os.system('say "Обработка данных завершена!"')

In [ ]:
cities = pd.DataFrame(df_locations_preproc[['city', 'id']])
df_locations_preproc.set_index('id')

In [ ]:
df_locations_preproc.to_csv('cities.csv')

In [11]:
#cities = pd.read_csv('cities.csv', delimiter=',', encoding='utf8')

In [ ]:
cities_train = cities[:1748890]
cities_test = cities[1748890:]

In [12]:
df_train_preproc = pd.concat([df_train_preproc, cities_train], axis=1)
df_train_preproc

,id,name,description,price,category,subcategory,id_,fields_categs,fields_adds,text,text_morph,Unnamed: 0,city,id
0,3edeb34cf93f490ff760af85,Сумка DG,NaN,199900.0,9,914,0,Женский гардероб Аксессуары,Тип Сумки Цвет Черный,сумка,[сумка],0,Moscow,3edeb34cf93f490ff760af85
1,c98febd50dad3cc0ffc86085,Комплект,8-12 лет,35000.0,22,2202,1,Детский гардероб Аксессуары,Тип Пол Цвет,комплект 8-12,[комплект],1,Moscow,c98febd50dad3cc0ffc86085
2,ade01e13912a46a99134cc75,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",30000.0,22,2204,2,Детский гардероб Верхняя одежда,Тип Пол Размер (рост) Цвет Сезон,пальтишко демисезонное девочку состояние хорошее,"[пальтишко, демисезонный, девочка, состояние, ...",2,Moscow,ade01e13912a46a99134cc75
3,ab3e6941c11304c1519aef75,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",80000.0,22,2209,3,Детский гардероб Обувь,Тип Размер Пол Цвет Сезон,attipas размер-135mm euro-22 прочная мягкая ск...,"[размер, прочный, мягкий, скользящий, обувь, м...",3,Moscow,ab3e6941c11304c1519aef75
4,252452a91c944a22c276d995,Жилет теплый,NaN,50000.0,22,2204,5,Детский гардероб Верхняя одежда,Тип Пол Размер (рост) Цвет Сезон,жилет теплый,"[жилет, тёплый]",5,Leningrad,252452a91c944a22c276d995
5,364f7659a4e9ba6c53e35495,"Босоножки, сабо","Новые, текстильные босоножки, 37 размер в разм...",30000.0,9,902,6,Женский гардероб Обувь,Тип Размер Цвет Сезон,босоножки сабо новые текстильные босоножки раз...,"[босоножка, сабо, новое, текстильный, босоножк...",6,Nizjnij Novgorod,364f7659a4e9ba6c53e35495
6,c3d87065e4dbc080f400bb85,Жесткий диск для ноутбука Seagate 320gb,Seagate Модель St320lt020 HDD для ноутбуков ...,150000.0,15,1508,7,Компьютерная техника Комплектующие и запчасти,Тип Устройство,жесткий диск ноутбука seagate 320gb seagate мо...,"[жёсткий, диск, ноутбук, модель, ноутбук, ёмко...",7,Voronezj,c3d87065e4dbc080f400bb85
7,22e6a518d53f490f6ce7fd85,Ковер 324 на 240,"Состояние отличное,без дефектов",200000.0,4,403,8,Для дома и дачи Текстиль и ковры,Тип Ковры,ковер состояние отличное дефектов,"[ковёр, состояние, отличный, дефект]",8,Stavropol'skiy,22e6a518d53f490f6ce7fd85
8,c6bd550b7f1304c1f7fda485,Кенгуру,"Продам кенгуру 350 , ванночка с горкой 250",35000.0,3,301,10,Детские товары Другое,NaN,кенгуру продам кенгуру ванночка горкой,"[кенгуру, продать, кенгуру, ванночка, горка]",10,Samara,c6bd550b7f1304c1f7fda485
9,117b885a3cf9554042ae2285,Машинка автомат,NaN,350000.0,2,208,11,Бытовая техника Стиральные машины,Тип Марка,машинка автомат,"[машинка, автомат]",11,Altai Krai,117b885a3cf9554042ae2285


In [13]:
df_test_preproc = pd.merge(df_test_preproc, cities_test, left_on='id', right_on='id')
df_test_preproc

,id,name,description,category,subcategory,id_,fields_categs,fields_adds,text,text_morph,Unnamed: 0,city
0,285ea2e9935ccdeb8378c6a5,Стремянка,"Стремянка трехсекционная,3×4",6,603,4,Ремонт и строительство Ручные инструменты,Тип,стремянка стремянка трехсекционная,"[стремянка, стремянка, трехсекционный]",4,Bashkortostan
1,adfb73820bbb831257df6e95,Плита электрическая,Плита ЭВИ 5120. Работают 3 конфорки и духовка....,2,203,9,Бытовая техника Плиты и духовые шкафы,Тип,плита электрическая плита 5120 работают конфор...,"[плита, электрический, плита, работать, конфор...",9,St.-Petersburg
2,783025601c36202f633fc6a5,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,Все лоты привезены с площадок и аукционов Япон...,1,116,15,Транспорт и автотовары Диски,"Тип Литые Ширина обода, дюймов 7 Диаметр, дюйм...",диски rays mazdaspeed touring 5х114 лоты приве...,"[диск, лот, площадка, аукцион, япония, пробег,...",15,Moskovskaya
3,2f0cd2d2e15dc90afd847f95,Batman Archam Knight,NaN,10,1009,19,Хобби и развлечения Игры для приставок и ПК,Платформа PlayStation 4 Жанр Драки,batman archam knight,[],19,Moscow
4,5c23a37902855a20172845a5,Ледобур для зимней рыбалки,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...,11,1104,20,Спорт и отдых Охота и рыбалка,Тип Ледобуры,ледобур зимней рыбалки отдаю ледобур времён ди...,"[ледобур, зимний, рыбалка, отдавать, ледобур, ...",20,St.-Petersburg
5,7b5ee55fc6ab657218c315a5,Sony плэйстейшен 2,Сонька в идеальном состоянии. 9 Дисков: CALL o...,10,1002,21,Хобби и развлечения Игровые приставки,Тип Приставки Платформа PlayStation 2,sony плэйстейшен сонька идеальном состоянии ди...,"[плэйстейшня, сонька, идеальный, состояние, ди...",21,Moskovskaya
6,7f2526891b18fff0b613e2a5,iPhone 4s 16Gb,"На запчасти или под восстановление, иногда син...",12,1202,24,Телефоны и планшеты Мобильные телефоны,Тип Марка Apple Операционная система iOS Диаго...,iphone 16gb запчасти восстановление иногда син...,"[запчасть, восстановление, синеть, пропадать, ...",24,Udmurtiya
7,8a36445a546c1e26657922a5,Манеж,Компактный удобный складывается в виде трости ...,3,317,28,Детские товары Обустройство детской,Тип Манежи,манеж компактный удобный складывается виде тро...,"[манеж, компактный, удобный, складываться, вид...",28,Tjumen
8,8307b793f4855a2010378f95,Оригинальный костюм жакет и юбка Olar Collect....,Продам костюм фирмы OLAR Collection S-44 рос. ...,9,908,29,Женский гардероб Пиджаки и костюмы,Тип Костюмы с юбкой Размер 42-44 (S) Цвет Серый,оригинальный костюм жакет юбка olar collect пр...,"[оригинальный, костюм, жакет, юбка, продать, к...",29,Moscow
9,b564c2b6763f490fe8fbb7a5,Теплые зимние сапоги 37 размер,Состояние очень хорошее.,9,902,31,Женский гардероб Обувь,Тип Сапоги Размер 37 Цвет Черный Сезон Зима,теплые зимние сапоги размер состояние очень хо...,"[тёплый, зимний, сапог, размер, состояние, хор...",31,Ivanovo


### Обучение

In [14]:
df_train_preproc['fields_categs'] = df_train_preproc['fields_categs'].fillna('Прочие')
#df_train_preproc['fields_adds'] = df_train_preproc['fields_adds'].fillna('')
df_train_preproc['city'] = df_train_preproc['city'].fillna('Караганда')

df_test_preproc['fields_categs'] = df_test_preproc['fields_categs'].fillna('Прочие')
#df_test_preproc['fields_adds'] = df_test_preproc['fields_adds'].fillna('')
df_test_preproc['city'] = df_test_preproc['city'].fillna('Караганда')

In [16]:
df_train_preproc['text_morph_str'] = df_train_preproc['text_morph'].map(lambda x: ' '.join(x))
df_test_preproc['text_morph_str'] = df_test_preproc['text_morph'].map(lambda x: ' '.join(x))

df_train_preproc['X'] = df_train_preproc['text_morph_str'] + ' ' + df_train_preproc['fields_categs'] + ' ' + df_train_preproc['city']
df_test_preproc['X'] = df_test_preproc['text_morph_str'] + ' ' + df_test_preproc['fields_categs'] + ' ' + df_test_preproc['city']

In [96]:
df_train_preproc['fields_adds'].isnull().values.any()

False

In [98]:
df_train_preproc

,id,name,description,price,category,subcategory,id_,fields_categs,fields_adds,text,text_morph,Unnamed: 0,city,id,text_morph_str,X
0,3edeb34cf93f490ff760af85,Сумка DG,NaN,1999.0,9,914,0,Женский гардероб Аксессуары,Тип Сумки Цвет Черный,сумка,[сумка],0,Moscow,3edeb34cf93f490ff760af85,сумка,сумка Женский гардероб Аксессуары Moscow
1,c98febd50dad3cc0ffc86085,Комплект,8-12 лет,350.0,22,2202,1,Детский гардероб Аксессуары,Тип Пол Цвет,комплект 8-12,[комплект],1,Moscow,c98febd50dad3cc0ffc86085,комплект,комплект Детский гардероб Аксессуары Moscow
2,ade01e13912a46a99134cc75,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",300.0,22,2204,2,Детский гардероб Верхняя одежда,Тип Пол Размер (рост) Цвет Сезон,пальтишко демисезонное девочку состояние хорошее,"[пальтишко, демисезонный, девочка, состояние, ...",2,Moscow,ade01e13912a46a99134cc75,пальтишко демисезонный девочка состояние хороший,пальтишко демисезонный девочка состояние хорош...
3,ab3e6941c11304c1519aef75,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",800.0,22,2209,3,Детский гардероб Обувь,Тип Размер Пол Цвет Сезон,attipas размер-135mm euro-22 прочная мягкая ск...,"[размер, прочный, мягкий, скользящий, обувь, м...",3,Moscow,ab3e6941c11304c1519aef75,размер прочный мягкий скользящий обувь маленьк...,размер прочный мягкий скользящий обувь маленьк...
4,252452a91c944a22c276d995,Жилет теплый,NaN,500.0,22,2204,5,Детский гардероб Верхняя одежда,Тип Пол Размер (рост) Цвет Сезон,жилет теплый,"[жилет, тёплый]",5,Leningrad,252452a91c944a22c276d995,жилет тёплый,жилет тёплый Детский гардероб Верхняя одежда L...
5,364f7659a4e9ba6c53e35495,"Босоножки, сабо","Новые, текстильные босоножки, 37 размер в разм...",300.0,9,902,6,Женский гардероб Обувь,Тип Размер Цвет Сезон,босоножки сабо новые текстильные босоножки раз...,"[босоножка, сабо, новое, текстильный, босоножк...",6,Nizjnij Novgorod,364f7659a4e9ba6c53e35495,босоножка сабо новое текстильный босоножка раз...,босоножка сабо новое текстильный босоножка раз...
6,c3d87065e4dbc080f400bb85,Жесткий диск для ноутбука Seagate 320gb,Seagate Модель St320lt020 HDD для ноутбуков ...,1500.0,15,1508,7,Компьютерная техника Комплектующие и запчасти,Тип Устройство,жесткий диск ноутбука seagate 320gb seagate мо...,"[жёсткий, диск, ноутбук, модель, ноутбук, ёмко...",7,Voronezj,c3d87065e4dbc080f400bb85,жёсткий диск ноутбук модель ноутбук ёмкость на...,жёсткий диск ноутбук модель ноутбук ёмкость на...
7,22e6a518d53f490f6ce7fd85,Ковер 324 на 240,"Состояние отличное,без дефектов",2000.0,4,403,8,Для дома и дачи Текстиль и ковры,Тип Ковры,ковер состояние отличное дефектов,"[ковёр, состояние, отличный, дефект]",8,Stavropol'skiy,22e6a518d53f490f6ce7fd85,ковёр состояние отличный дефект,ковёр состояние отличный дефект Для дома и дач...
8,c6bd550b7f1304c1f7fda485,Кенгуру,"Продам кенгуру 350 , ванночка с горкой 250",350.0,3,301,10,Детские товары Другое,,кенгуру продам кенгуру ванночка горкой,"[кенгуру, продать, кенгуру, ванночка, горка]",10,Samara,c6bd550b7f1304c1f7fda485,кенгуру продать кенгуру ванночка горка,кенгуру продать кенгуру ванночка горка Детские...
9,117b885a3cf9554042ae2285,Машинка автомат,NaN,3500.0,2,208,11,Бытовая техника Стиральные машины,Тип Марка,машинка автомат,"[машинка, автомат]",11,Altai Krai,117b885a3cf9554042ae2285,машинка автомат,машинка автомат Бытовая техника Стиральные маш...


In [99]:
df_test_preproc

,id,name,description,category,subcategory,id_,fields_categs,fields_adds,text,text_morph,Unnamed: 0,city,text_morph_str,X
0,285ea2e9935ccdeb8378c6a5,Стремянка,"Стремянка трехсекционная,3×4",6,603,4,Ремонт и строительство Ручные инструменты,Тип,стремянка стремянка трехсекционная,"[стремянка, стремянка, трехсекционный]",4,Bashkortostan,стремянка стремянка трехсекционный,стремянка стремянка трехсекционный Ремонт и ст...
1,adfb73820bbb831257df6e95,Плита электрическая,Плита ЭВИ 5120. Работают 3 конфорки и духовка....,2,203,9,Бытовая техника Плиты и духовые шкафы,Тип,плита электрическая плита 5120 работают конфор...,"[плита, электрический, плита, работать, конфор...",9,St.-Petersburg,плита электрический плита работать конфорка ду...,плита электрический плита работать конфорка ду...
2,783025601c36202f633fc6a5,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,Все лоты привезены с площадок и аукционов Япон...,1,116,15,Транспорт и автотовары Диски,"Тип Литые Ширина обода, дюймов 7 Диаметр, дюйм...",диски rays mazdaspeed touring 5х114 лоты приве...,"[диск, лот, площадка, аукцион, япония, пробег,...",15,Moskovskaya,диск лот площадка аукцион япония пробег наличи...,диск лот площадка аукцион япония пробег наличи...
3,2f0cd2d2e15dc90afd847f95,Batman Archam Knight,NaN,10,1009,19,Хобби и развлечения Игры для приставок и ПК,Платформа PlayStation 4 Жанр Драки,batman archam knight,[],19,Moscow,,Хобби и развлечения Игры для приставок и ПК M...
4,5c23a37902855a20172845a5,Ледобур для зимней рыбалки,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...,11,1104,20,Спорт и отдых Охота и рыбалка,Тип Ледобуры,ледобур зимней рыбалки отдаю ледобур времён ди...,"[ледобур, зимний, рыбалка, отдавать, ледобур, ...",20,St.-Petersburg,ледобур зимний рыбалка отдавать ледобур время ...,ледобур зимний рыбалка отдавать ледобур время ...
5,7b5ee55fc6ab657218c315a5,Sony плэйстейшен 2,Сонька в идеальном состоянии. 9 Дисков: CALL o...,10,1002,21,Хобби и развлечения Игровые приставки,Тип Приставки Платформа PlayStation 2,sony плэйстейшен сонька идеальном состоянии ди...,"[плэйстейшня, сонька, идеальный, состояние, ди...",21,Moskovskaya,плэйстейшня сонька идеальный состояние диск ос...,плэйстейшня сонька идеальный состояние диск ос...
6,7f2526891b18fff0b613e2a5,iPhone 4s 16Gb,"На запчасти или под восстановление, иногда син...",12,1202,24,Телефоны и планшеты Мобильные телефоны,Тип Марка Apple Операционная система iOS Диаго...,iphone 16gb запчасти восстановление иногда син...,"[запчасть, восстановление, синеть, пропадать, ...",24,Udmurtiya,запчасть восстановление синеть пропадать звук ...,запчасть восстановление синеть пропадать звук ...
7,8a36445a546c1e26657922a5,Манеж,Компактный удобный складывается в виде трости ...,3,317,28,Детские товары Обустройство детской,Тип Манежи,манеж компактный удобный складывается виде тро...,"[манеж, компактный, удобный, складываться, вид...",28,Tjumen,манеж компактный удобный складываться вид трос...,манеж компактный удобный складываться вид трос...
8,8307b793f4855a2010378f95,Оригинальный костюм жакет и юбка Olar Collect....,Продам костюм фирмы OLAR Collection S-44 рос. ...,9,908,29,Женский гардероб Пиджаки и костюмы,Тип Костюмы с юбкой Размер 42-44 (S) Цвет Серый,оригинальный костюм жакет юбка olar collect пр...,"[оригинальный, костюм, жакет, юбка, продать, к...",29,Moscow,оригинальный костюм жакет юбка продать костюм ...,оригинальный костюм жакет юбка продать костюм ...
9,b564c2b6763f490fe8fbb7a5,Теплые зимние сапоги 37 размер,Состояние очень хорошее.,9,902,31,Женский гардероб Обувь,Тип Сапоги Размер 37 Цвет Черный Сезон Зима,теплые зимние сапоги размер состояние очень хо...,"[тёплый, зимний, сапог, размер, состояние, хор...",31,Ivanovo,тёплый зимний сапог размер состояние хороший,тёплый зимний сапог размер состояние хороший Ж...


In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_train_preproc['X'], df_train_preproc['price'], test_size=0.8, random_state=123)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(349778,)
(349778,)
(1399112,)
(1399112,)


In [18]:
%%time

#counter = CountVectorizer(stop_words = stop_words, ngram_range = (1,2))
counter = TfidfVectorizer(min_df=1, analyzer='word', stop_words=stop_words, sublinear_tf=True)

model_RFR = Pipeline([
    ('cv', counter),
    ('lr', RandomForestRegressor(n_estimators=10))
])

model_RFR.fit(X_train, y_train)

predict_RFR = model_RFR.predict(X_test)

print(model_RFR.score(X_test, y_test))

os.system('say "Обучение завершено"')

-0.13585058473162737
CPU times: user 2h 2min 28s, sys: 1min 4s, total: 2h 3min 32s
Wall time: 3h 4min 37s


In [24]:
def rlmse_score(y_test, y_hat):
    return np.sqrt(np.mean((np.log(y_test + 1) - np.log(y_hat + 1))**2))

In [25]:
rlmse_score(y_test, predict_RFR)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


inf

### Получение результата

In [21]:
%%time
result_predict = model_RFR.predict(df_test_preproc['X'])

df_result = pd.DataFrame(data = {'id':df_test_preproc['id'], 'price':result_predict})

resFileName = 'result_'+ str(res_rlmse_score).replace('.','_') +'.csv'
df_result.to_csv(resFileName, sep=',', encoding='utf-8', index=False)
print(resFileName)

os.system('say "Работа завершена!"')

result_inf.csv
CPU times: user 26.3 s, sys: 1.91 s, total: 28.2 s
Wall time: 33.3 s


In [22]:
df_result.head(100)

# Test                      rlmse               real score
# only raw text             1.41430286515203    4.57311
# morphy                    1.4446533598156919  4.54545
# mystem                    1.4218726116168907  4.55375
# fields_categs+fields_adds 1.4671398488301333  4.56130
# fields_categs             1.4617336779885182  4.57998
# +location                 1.3826812586955728  4.54742

,id,price
0,285ea2e9935ccdeb8378c6a5,2.517000e+05
1,adfb73820bbb831257df6e95,1.920000e+05
2,783025601c36202f633fc6a5,1.739900e+06
3,2f0cd2d2e15dc90afd847f95,9.976088e+04
4,5c23a37902855a20172845a5,1.655000e+05
5,7b5ee55fc6ab657218c315a5,5.220000e+05
6,7f2526891b18fff0b613e2a5,5.899000e+04
7,8a36445a546c1e26657922a5,1.180000e+05
8,8307b793f4855a2010378f95,2.540000e+05
9,b564c2b6763f490fe8fbb7a5,5.550000e+04
